In [ ]:
import molsysmt as msm
import pyunitwizard as puw

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
molecular_system = msm.systems.demo['Barnase-Barstar']['barnase_barstar.msmpk']
molecular_system = msm.convert(molecular_system)

In [ ]:
msm.info(molecular_system, element='component')

In [ ]:
msm.molecular_mechanics.potential_energy_minimization(molecular_system)

In [ ]:
msm.molecular_mechanics.get_potential_energy(molecular_system, decomposition=True)

In [ ]:
U12 = msm.molecular_mechanics.get_non_bonded_potential_energy(molecular_system,
                                                              selection='molecule_index==0',
                                                              selection_2='molecule_index==1')

In [ ]:
U12

In [ ]:
%%time
U12_groups = msm.molecular_mechanics.get_non_bonded_potential_energy(molecular_system,
                                                              selection='all in groups of molecule_index==0',
                                                              selection_2='all in groups of molecule_index==1')

In [ ]:
vtop = puw.get_value(max(abs(U12_groups.min()), abs(U12_groups.max())))

plt.imshow(U12_groups, origin='lower', cmap='bwr', vmin=-vtop, vmax=vtop)
plt.colorbar()
plt.show()

In [ ]:
mask = np.absolute(msm.pyunitwizard.get_value(U12_groups))>10.0
np.sum(mask)

In [ ]:
mask2 = msm.pyunitwizard.get_value(U12_groups)<-10.0
np.sum(mask2)

In [ ]:
np.sum(U12_groups[mask])/U12

In [ ]:
U12_1_groups= U12_groups.sum(axis=1)
U12_2_groups= U12_groups.sum(axis=0)

plt.bar(np.arange(U12_1_groups.shape[0]), msm.pyunitwizard.get_value(U12_1_groups))
plt.show()

plt.bar(np.arange(U12_2_groups.shape[0]), msm.pyunitwizard.get_value(U12_2_groups))
plt.show()

In [ ]:
distance = msm.structure.get_distances(molecular_system, selection='all in groups of molecule_index==0',
                 selection_2='all in groups of molecule_index==1')

In [ ]:
plt.scatter(distance.flatten(), U12_groups.flatten(), s=1.0)
plt.ylim([-100.0, 100.0])
plt.show()

In [ ]:
aux = [ii for ii in msm.pyunitwizard.get_value(U12_1_groups)]
aux += [ii for ii in msm.pyunitwizard.get_value(U12_2_groups)]
aux = np.array(aux)
max_abs_val = max(abs(aux.min()), abs(aux.max()))

In [ ]:
view = msm.view(molecular_system)
view.clear()
view.add_cartoon(selection='all')
msm.thirds.nglview.color_by_value(view, aux, min_value= -max_abs_val, max_value= max_abs_val, cmap='bwr')
view